In [15]:
# https://platform.olimpiada-ai.ro/problems/56

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision.transforms import v2

In [16]:
train = pd.read_csv("/kaggle/input/pokemon/train.csv")
test = pd.read_csv("/kaggle/input/pokemon/test.csv")

train.shape, test.shape

((647, 3), (162, 2))

In [17]:
train, valid = train_test_split(train, test_size=0.2, random_state=42, stratify=train['Type'])

train.shape, valid.shape

((517, 3), (130, 3))

In [18]:
from transformers import pipeline

pipe = pipeline("image-feature-extraction", model="google/vit-base-patch16-224-in21k")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cpu


In [29]:
train_embeddings = []
valid_embeddings = []
test_embeddings = []

for i, row in tqdm(train.iterrows(), total=len(train)):
    emb = pipe(os.path.join('/kaggle/input/pokemon', row['ImagePath']), return_tensors='np')[:, 0, :][0]
    emb = emb / np.linalg.norm(emb)
    train_embeddings.append(emb)

for i, row in tqdm(valid.iterrows(), total=len(valid)):
    emb = pipe(os.path.join('/kaggle/input/pokemon', row['ImagePath']), return_tensors='np')[:, 0, :][0]
    emb = emb / np.linalg.norm(emb)
    valid_embeddings.append(emb)

for i, row in tqdm(test.iterrows(), total=len(test)):
    emb = pipe(os.path.join('/kaggle/input/pokemon', row['ImagePath']), return_tensors='np')[:, 0, :][0]
    emb = emb / np.linalg.norm(emb)
    test_embeddings.append(emb)

train_embeddings = np.stack(train_embeddings)
valid_embeddings = np.stack(valid_embeddings)
test_embeddings = np.stack(test_embeddings)

  0%|          | 0/517 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/162 [00:00<?, ?it/s]

In [132]:
from sklearn.linear_model import RidgeClassifier

model = RidgeClassifier(alpha=1.0)

model.fit(train_embeddings, train['Type'])

RidgeClassifier()

In [133]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(valid_embeddings)

score = accuracy_score(y_pred, valid['Type'])

print(f'Score: {score:.5f}')

Score: 0.28462


In [134]:
model = RidgeClassifier(alpha=1.0)

model.fit(np.concatenate([train_embeddings, valid_embeddings]), pd.concat([train['Type'], valid['Type']]))

RidgeClassifier()

In [135]:
y_pred = model.predict(test_embeddings)

subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'Type': y_pred
})

subm.to_csv("submission.csv", index=False)

subm['Type'].value_counts()

Type
Water       42
Normal      33
Bug         23
Grass       19
Fire        14
Steel        6
Rock         5
Psychic      5
Poison       4
Electric     3
Ghost        2
Ground       2
Dark         2
Dragon       1
Fairy        1
Name: count, dtype: int64